<a href="https://colab.research.google.com/github/socrisogo/bc_social_media_productivity/blob/main/Proyecto_TTECH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Este es el formato para el proyecto TTECH


# Presentación

# Justificación

# Ingestión de datos

# Preparación y limpieza

# Analítica descriptiva

# Análisis de negocio